In [ ]:
# 설치
! pip install smolagents
# 마지막 릴리스 대신 소스에서 설치하려면 위 명령을 주석 처리하고 다음 명령의 주석을 해제하십시오.
# ! pip install git+https://github.com/huggingface/smolagents.git

# 보안 코드 실행

> [!TIP]
> 에이전트 빌드가 처음이라면 먼저 [에이전트 소개](https://huggingface.co/docs/smolagents/main/ko/tutorials/../conceptual_guides/intro_agents)와 [smolagents 둘러보기](https://huggingface.co/docs/smolagents/main/ko/tutorials/../guided_tour)를 읽어보세요.

### 코드 에이전트

[여러](https://huggingface.co/papers/2402.01030) [연구](https://huggingface.co/papers/2411.01747) [논문](https://huggingface.co/papers/2401.00812)에서는 LLM이 코드에 작업(도구 호출)을 작성하는 것이 현재 업계 표준인 도구 호출 형식(도구 이름과 사용할 인수의 JSON으로 작업을 작성하는 다양한 방식)보다 훨씬 낫다는 것을 보여주었습니다.

코드가 더 나은 이유는 무엇일까요? 글쎄요, 컴퓨터가 수행하는 작업을 표현하는 데 탁월하도록 코드 언어를 특별히 만들었기 때문입니다. JSON 스니펫이 더 나은 방법이었다면 이 패키지는 JSON 스니펫으로 작성되었을 것이고 악마는 우리를 비웃었을 것입니다.

코드는 컴퓨터에서 작업을 표현하는 더 나은 방법일 뿐입니다. 다음과 같은 장점이 있습니다.
- **구성 가능성:** 파이썬 함수를 정의하는 것처럼 JSON 작업을 서로 중첩하거나 나중에 다시 사용할 JSON 작업 집합을 정의할 수 있을까요?
- **개체 관리:** JSON에서 `generate_image`와 같은 작업의 출력을 어떻게 저장합니까?
- **일반성:** 코드는 컴퓨터가 할 수 있는 모든 것을 간단하게 표현하도록 만들어졌습니다.
- **LLM 훈련 코퍼스에서의 표현:** LLM 훈련 코퍼스에 이미 많은 양질의 작업이 포함되어 있다는 하늘의 축복을 활용하지 않을 이유가 있을까요?

이는 [실행 가능한 코드 작업이 더 나은 LLM 에이전트를 유도한다](https://huggingface.co/papers/2402.01030)에서 가져온 아래 그림에 설명되어 있습니다.

<img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/code_vs_json_actions.png">

이것이 바로 우리가 이 경우 파이썬 에이전트인 코드 에이전트를 제안하는 데 중점을 두는 이유이며, 이는 보안 파이썬 인터프리터를 구축하는 데 더 많은 노력을 기울이는 것을 의미합니다.

### 로컬 코드 실행??

기본적으로 `CodeAgent`는 환경에서 LLM 생성 코드를 실행합니다.

이는 본질적으로 위험하며 LLM 생성 코드는 환경에 해로울 수 있습니다.

악성 코드 실행은 여러 가지 방식으로 발생할 수 있습니다.
- **단순 LLM 오류:** LLM은 아직 완벽과는 거리가 멀며 도움이 되려고 시도하는 동안 의도치 않게 유해한 명령을 생성할 수 있습니다. 이 위험은 낮지만 LLM이 잠재적으로 위험한 코드를 실행하려고 시도한 사례가 관찰되었습니다.  
- **공급망 공격:** 신뢰할 수 없거나 손상된 LLM을 실행하면 시스템이 유해한 코드 생성에 노출될 수 있습니다. 보안 추론 인프라에서 잘 알려진 모델을 사용할 때 이 위험은 극히 낮지만 이론적으로는 가능성이 남아 있습니다.  
- **프롬프트 주입:** 웹을 탐색하는 에이전트가 유해한 지침이 포함된 악성 웹사이트에 도달하여 에이전트의 메모리에 공격을 주입할 수 있습니다.
- **공개적으로 액세스 가능한 에이전트 악용:** 공개적으로 노출된 에이전트는 악의적인 행위자가 유해한 코드를 실행하는 데 오용될 수 있습니다. 공격자는 에이전트의 실행 기능을 악용하기 위해 적대적인 입력을 만들어 의도하지 않은 결과를 초래할 수 있습니다.
악성 코드가 우발적이든 의도적이든 실행되면 파일 시스템을 손상시키고 로컬 또는 클라우드 기반 리소스를 악용하며 API 서비스를 남용하고 네트워크 보안을 손상시킬 수도 있습니다.

[에이전시 스펙트럼](https://huggingface.co/docs/smolagents/main/ko/tutorials/../conceptual_guides/intro_agents)에서 코드 에이전트는 다른 덜 에이전트적인 설정보다 시스템에서 LLM에 훨씬 더 높은 에이전시를 부여한다고 주장할 수 있습니다. 이는 더 높은 위험과 밀접하게 관련됩니다.

따라서 보안에 매우 유의해야 합니다.

안전성을 향상시키기 위해 더 높은 설정 비용으로 더 높은 수준의 보안을 제안하는 다양한 조치를 제안합니다.

어떤 솔루션도 100% 안전하지 않다는 점을 명심하십시오.

<img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/smolagents/code_execution_safety_diagram.png">

### 로컬 Python 실행기

첫 번째 보안 계층을 추가하기 위해 `smolagents`의 코드 실행은 바닐라 Python 인터프리터에서 수행되지 않습니다.
처음부터 더 안전한 `LocalPythonExecutor`를 다시 빌드했습니다.

정확히 말하면 이 인터프리터는 코드에서 추상 구문 트리(AST)를 로드하고 특정 규칙을 항상 따르도록 하면서 작업별로 실행합니다.
- 기본적으로 사용자가 명시적으로 권한 부여 목록에 추가하지 않는 한 가져오기는 허용되지 않습니다.
- 또한 하위 모듈에 대한 액세스는 기본적으로 비활성화되어 있으며 각 모듈은 가져오기 목록에서 명시적으로 권한을 부여해야 하거나 예를 들어 `numpy.*`를 전달하여 `numpy`와 `numpy.random` 또는 `numpy.a.b`와 같은 모든 하위 패키지를 허용할 수 있습니다.
   - `random`과 같이 겉보기에 무해해 보이는 일부 패키지는 `random._os`와 같이 잠재적으로 유해한 하위 모듈에 대한 액세스를 제공할 수 있습니다.
- 처리된 기본 작업의 총 수는 무한 루프 및 리소스 팽창을 방지하기 위해 제한됩니다.
- 사용자 지정 인터프리터에서 명시적으로 정의되지 않은 모든 작업은 오류를 발생시킵니다.

다음과 같이 이러한 안전 장치를 시도해 볼 수 있습니다.

In [ ]:
from smolagents.local_python_executor import LocalPythonExecutor

# 사용자 지정 실행기 설정, "numpy" 패키지 권한 부여
custom_executor = LocalPythonExecutor(["numpy"])

# 오류를 예쁘게 인쇄하기 위한 유틸리티
def run_capture_exception(command: str):
    try:
        custom_executor(harmful_command)
    except Exception as e:
        print("오류:\n", e)

# 정의되지 않은 명령은 작동하지 않음
harmful_command="!echo 잘못된 명령"
run_capture_exception(harmful_command)
# >>> 오류: 잘못된 구문 (<알 수 없음>, 1행)


# os와 같은 가져오기는 `additional_authorized_imports`에 명시적으로 추가되지 않는 한 수행되지 않음
harmful_command="import os; exit_code = os.system(\"echo 잘못된 명령\")"
run_capture_exception(harmful_command)
# >>> 오류: 다음으로 인해 'import os' 줄에서 코드 실행 실패: InterpreterError: os 가져오기가 허용되지 않습니다. 승인된 가져오기는 ['statistics', 'numpy', 'itertools', 'time', 'queue', 'collections', 'math', 'random', 're', 'datetime', 'stat', 'unicodedata']입니다.

# 승인된 가져오기에서도 잠재적으로 유해한 패키지는 가져오지 않음
harmful_command="import random; random._os.system('echo 잘못된 명령')"
run_capture_exception(harmful_command)
# >>> 오류: 다음으로 인해 'random._os.system('echo 잘못된 명령')' 줄에서 코드 실행 실패: InterpreterError: os 모듈에 대한 액세스 금지

# 무한 루프는 N번 작업 후 중단됨
harmful_command="""
while True:
    pass
"""
run_capture_exception(harmful_command)
# >>> 오류: 다음으로 인해 'while True: pass' 줄에서 코드 실행 실패: InterpreterError: While 루프에서 최대 1000000회 반복 초과

이러한 안전 장치는 인터프리터를 더 안전하게 만듭니다.
다양한 사용 사례에서 사용했으며 환경에 손상을 입힌 적이 없습니다.

> [!WARNING]
> 어떤 로컬 파이썬 샌드박스도 완전히 안전할 수는 없다는 것을 이해하는 것이 중요합니다. 우리 인터프리터는 표준 Python 인터프리터보다 훨씬 향상된 안전성을 제공하지만, 단호한 공격자나 미세 조정된 악성 LLM이 취약점을 찾아 환경에 잠재적으로 해를 끼칠 가능성은 여전히 존재합니다.
> 
> 예를 들어, 이미지를 처리하기 위해 `Pillow`와 같은 패키지를 허용한 경우 LLM은 하드 드라이브를 채우기 위해 수천 개의 대용량 이미지 파일을 생성하는 코드를 생성할 수 있습니다. 다른 고급 탈출 기술은 승인된 패키지의 더 깊은 취약점을 악용할 수 있습니다.
> 
> 로컬 환경에서 LLM 생성 코드를 실행하는 것은 항상 약간의 고유한 위험을 수반합니다. LLM 생성 코드를 진정으로 강력한 보안 격리로 실행하는 유일한 방법은 아래에 자세히 설명된 E2B 또는 Docker와 같은 원격 실행 옵션을 사용하는 것입니다.

신뢰할 수 있는 추론 공급자의 잘 알려진 LLM을 사용할 때 악성 공격의 위험은 낮지만 0은 아닙니다.
보안 수준이 높은 애플리케이션이나 신뢰도가 낮은 모델을 사용하는 경우 원격 실행 샌드박스를 사용하는 것을 고려해야 합니다.

## 보안 코드 실행을 위한 샌드박스 접근 방식

코드를 실행하는 AI 에이전트로 작업할 때는 보안이 가장 중요합니다. smolagents에서 코드를 샌드박싱하는 데는 두 가지 주요 접근 방식이 있으며, 각각 다른 보안 속성과 기능을 가지고 있습니다.


![샌드박스 접근 방식 비교](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/smolagents/remote_execution.png)

1. **샌드박스에서 개별 코드 스니펫 실행**: 이 접근 방식(다이어그램 왼쪽)은 에이전트 생성 Python 코드 스니펫만 샌드박스에서 실행하고 나머지 에이전트 시스템은 로컬 환경에 유지합니다. `executor_type="e2b"` 또는 `executor_type="docker"`를 사용하여 설정하기가 더 간단하지만 다중 에이전트를 지원하지 않으며 여전히 환경과 샌드박스 간에 상태 데이터를 전달해야 합니다.

2. **샌드박스에서 전체 에이전트 시스템 실행**: 이 접근 방식(다이어그램 오른쪽)은 에이전트, 모델 및 도구를 포함한 전체 에이전트 시스템을 샌드박스 환경 내에서 실행합니다. 이렇게 하면 더 나은 격리를 제공하지만 더 많은 수동 설정이 필요하며 민감한 자격 증명(예: API 키)을 샌드박스 환경에 전달해야 할 수 있습니다.

이 가이드에서는 에이전트 애플리케이션에 대한 두 가지 유형의 샌드박스 접근 방식을 설정하고 사용하는 방법을 설명합니다.

### E2B 설정

#### 설치

1. [e2b.dev](https://e2b.dev)에서 E2B 계정 만들기
2. 필요한 패키지 설치:
```bash
pip install 'smolagents[e2b]'
```

#### E2B에서 에이전트 실행: 빠른 시작

E2B 샌드박스를 사용하는 간단한 방법을 제공합니다. 에이전트 초기화에 `executor_type="e2b"`를 추가하기만 하면 됩니다.

In [ ]:
from smolagents import InferenceClientModel, CodeAgent

agent = CodeAgent(model=InferenceClientModel(), tools=[], executor_type="e2b")

with agent:
    agent.run("100번째 피보나치 수를 알려주시겠어요?")

> [!TIP]
> 에이전트를 컨텍스트 관리자(`with` 문 사용)로 사용하면 에이전트가 작업을 완료한 직후 E2B 샌드박스가 정리되도록 보장합니다.
> 또는 에이전트의 `cleanup()` 메서드를 수동으로 호출할 수 있습니다.

이 솔루션은 각 `agent.run()` 시작 시 에이전트 상태를 서버로 보냅니다.
그런 다음 모델은 로컬 환경에서 호출되지만 생성된 코드는 실행을 위해 샌드박스로 전송되고 출력만 반환됩니다.

이는 아래 그림에 설명되어 있습니다.

<p align="center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/smolagents/sandboxed_execution.png" alt="샌드박스 코드 실행" width=60% max-width=500px>
</p>

그러나 [관리되는 에이전트](https://huggingface.co/docs/smolagents/main/ko/tutorials/../examples/multiagents)에 대한 모든 호출에는 모델 호출이 필요하므로 원격 샌드박스로 비밀을 전송하지 않기 때문에 모델 호출에 자격 증명이 부족합니다.
따라서 이 솔루션은 더 복잡한 다중 에이전트 설정에서는 (아직) 작동하지 않습니다.

#### E2B에서 에이전트 실행: 다중 에이전트

E2B 샌드박스에서 다중 에이전트를 사용하려면 E2B 내에서 에이전트를 완전히 실행해야 합니다.

방법은 다음과 같습니다.

In [ ]:
from e2b_code_interpreter import Sandbox
import os

# 샌드박스 만들기
sandbox = Sandbox()

# 필요한 패키지 설치
sandbox.commands.run("pip install smolagents")

def run_code_raise_errors(sandbox, code: str, verbose: bool = False) -> str:
    execution = sandbox.run_code(
        code,
        envs={'HF_TOKEN': os.getenv('HF_TOKEN')}
    )
    if execution.error:
        execution_logs = "\n".join([str(log) for log in execution.logs.stdout])
        logs = execution_logs
        logs += execution.error.traceback
        raise ValueError(logs)
    return "\n".join([str(log) for log in execution.logs.stdout])

# 에이전트 애플리케이션 정의
agent_code = """
import os
from smolagents import CodeAgent, InferenceClientModel

# 에이전트 초기화
agent = CodeAgent(
    model=InferenceClientModel(token=os.getenv("HF_TOKEN"), provider="together"),
    tools=[],
    name="coder_agent",
    description="이 에이전트는 코드를 사용하여 어려운 알고리즘 문제를 처리합니다."
)

manager_agent = CodeAgent(
    model=InferenceClientModel(token=os.getenv("HF_TOKEN"), provider="together"),
    tools=[],
    managed_agents=[agent],
)

# 에이전트 실행
response = manager_agent.run("20번째 피보나치 수는 무엇인가요?")
print(response)
"""

# 샌드박스에서 에이전트 코드 실행
execution_logs = run_code_raise_errors(sandbox, agent_code)
print(execution_logs)

### Docker 설정

#### 설치

1. [시스템에 Docker 설치](https://docs.docker.com/get-started/get-docker/)
2. 필요한 패키지 설치:
```bash
pip install 'smolagents[docker]'
```

#### Docker에서 에이전트 실행: 빠른 시작

위의 E2B 샌드박스와 유사하게 Docker를 빠르게 시작하려면 에이전트 초기화에 `executor_type="docker"`를 추가하기만 하면 됩니다.

In [ ]:
from smolagents import InferenceClientModel, CodeAgent

agent = CodeAgent(model=InferenceClientModel(), tools=[], executor_type="docker")

with agent:
    agent.run("100번째 피보나치 수를 알려주시겠어요?")

> [!TIP]
> 에이전트를 컨텍스트 관리자(`with` 문 사용)로 사용하면 에이전트가 작업을 완료한 직후 Docker 컨테이너가 정리되도록 보장합니다.
> 또는 에이전트의 `cleanup()` 메서드를 수동으로 호출할 수 있습니다.

#### 고급 Docker 사용법

Docker에서 다중 에이전트 시스템을 실행하려면 샌드박스에서 사용자 지정 인터프리터를 설정해야 합니다.

Dockerfile을 설정하는 방법은 다음과 같습니다.

```dockerfile
FROM python:3.10-bullseye

# 빌드 종속성 설치
RUN apt-get update && \
    apt-get install -y --no-install-recommends \
        build-essential \
        python3-dev && \
    pip install --no-cache-dir --upgrade pip && \
    pip install --no-cache-dir smolagents && \
    apt-get clean && \
    rm -rf /var/lib/apt/lists/*

# 작업 디렉터리 설정
WORKDIR /app

# 제한된 권한으로 실행
USER nobody

# 기본 명령
CMD ["python", "-c", "print('컨테이너 준비 완료')"]
```

코드를 실행할 샌드박스 관리자 만들기:

In [ ]:
import docker
import os
from typing import Optional

class DockerSandbox:
    def __init__(self):
        self.client = docker.from_env()
        self.container = None

    def create_container(self):
        try:
            image, build_logs = self.client.images.build(
                path=".",
                tag="agent-sandbox",
                rm=True,
                forcerm=True,
                buildargs={},
                # decode=True
            )
        except docker.errors.BuildError as e:
            print("빌드 오류 로그:")
            for log in e.build_log:
                if 'stream' in log:
                    print(log['stream'].strip())
            raise

        # 보안 제약 조건 및 적절한 로깅으로 컨테이너 만들기
        self.container = self.client.containers.run(
            "agent-sandbox",
            command="tail -f /dev/null",  # 컨테이너 계속 실행
            detach=True,
            tty=True,
            mem_limit="512m",
            cpu_quota=50000,
            pids_limit=100,
            security_opt=["no-new-privileges"],
            cap_drop=["ALL"],
            environment={
                "HF_TOKEN": os.getenv("HF_TOKEN")
            },
        )

    def run_code(self, code: str) -> Optional[str]:
        if not self.container:
            self.create_container()

        # 컨테이너에서 코드 실행
        exec_result = self.container.exec_run(
            cmd=["python", "-c", code],
            user="nobody"
        )

        # 모든 출력 수집
        return exec_result.output.decode() if exec_result.output else None


    def cleanup(self):
        if self.container:
            try:
                self.container.stop()
            except docker.errors.NotFound:
                # 컨테이너가 이미 제거되었으므로 예상된 동작임
                pass
            except Exception as e:
                print(f"정리 중 오류 발생: {e}")
            finally:
                self.container = None  # 참조 지우기

# 사용 예시:
sandbox = DockerSandbox()

try:
    # 에이전트 코드 정의
    agent_code = """
import os
from smolagents import CodeAgent, InferenceClientModel

# 에이전트 초기화
agent = CodeAgent(
    model=InferenceClientModel(token=os.getenv("HF_TOKEN"), provider="together"),
    tools=[]
)

# 에이전트 실행
response = agent.run("20번째 피보나치 수는 무엇인가요?")
print(response)
"""

    # 샌드박스에서 코드 실행
    output = sandbox.run_code(agent_code)
    print(output)

finally:
    sandbox.cleanup()

### WebAssembly 설정

WebAssembly(Wasm)는 코드를 안전한 샌드박스 환경에서 실행할 수 있도록 하는 이진 명령어 형식입니다.
빠르고 효율적이며 안전하도록 설계되어 잠재적으로 신뢰할 수 없는 코드를 실행하는 데 탁월한 선택입니다.

`WasmExecutor`는 [Pyodide](https://pyodide.org/)와 [Deno](https://docs.deno.com/)를 사용합니다.

#### 설치

1. [시스템에 Deno 설치](https://docs.deno.com/runtime/getting_started/installation/)

#### WebAssembly에서 에이전트 실행: 빠른 시작

에이전트 초기화에 `executor_type="wasm"`을 전달하기만 하면 됩니다.

In [ ]:
from smolagents import InferenceClientModel, CodeAgent

agent = CodeAgent(model=InferenceClientModel(), tools=[], executor_type="wasm")

agent.run("100번째 피보나치 수를 알려주시겠어요?")

### 샌드박스 모범 사례

이러한 주요 사례는 E2B 및 Docker 샌드박스 모두에 적용됩니다.

- 리소스 관리
  - 메모리 및 CPU 제한 설정
  - 실행 시간 초과 구현
  - 리소스 사용량 모니터링
- 보안
  - 최소 권한으로 실행
  - 불필요한 네트워크 액세스 비활성화
  - 비밀에 환경 변수 사용
- 환경
  - 종속성을 최소한으로 유지
  - 고정된 패키지 버전 사용
  - 기본 이미지를 사용하는 경우 정기적으로 업데이트

- 정리
  - 특히 Docker 컨테이너의 경우 리소스를 제대로 정리하여 매달린 컨테이너가 리소스를 소모하지 않도록 항상 확인하십시오.

✨ 이러한 사례를 따르고 적절한 정리 절차를 구현하면 에이전트가 샌드박스 환경에서 안전하고 효율적으로 실행되도록 보장할 수 있습니다.

## 보안 접근 방식 비교

앞서 다이어그램에서 설명했듯이 두 샌드박싱 접근 방식 모두 다른 보안 영향을 미칩니다.

### 접근 방식 1: 샌드박스에서 코드 스니펫만 실행

- **장점**:
  - 간단한 매개변수(`executor_type="e2b"` 또는 `executor_type="docker"`)로 설정하기 쉬움
  - API 키를 샌드박스로 전송할 필요 없음
  - 로컬 환경에 대한 더 나은 보호
- **단점**:
  - 다중 에이전트(관리형 에이전트)를 지원하지 않음
  - 여전히 환경과 샌드박스 간에 상태를 전송해야 함
  - 특정 코드 실행으로 제한됨

### 접근 방식 2: 샌드박스에서 전체 에이전트 시스템 실행

- **장점**:
  - 다중 에이전트 지원
  - 전체 에이전트 시스템의 완전한 격리
  - 복잡한 에이전트 아키텍처에 더 유연함
- **단점**:
  - 더 많은 수동 설정 필요
  - 민감한 API 키를 샌드박스로 전송해야 할 수 있음
  - 더 복잡한 작업으로 인해 잠재적으로 더 높은 대기 시간

보안 요구 사항과 애플리케이션 요구 사항의 균형을 가장 잘 맞추는 접근 방식을 선택하십시오. 더 간단한 에이전트 아키텍처를 가진 대부분의 애플리케이션의 경우 접근 방식 1은 보안과 사용 편의성의 적절한 균형을 제공합니다. 완전한 격리가 필요한 더 복잡한 다중 에이전트 시스템의 경우 설정하는 데 더 많은 노력이 필요하지만 접근 방식 2는 더 나은 보안 보장을 제공합니다.